In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
abc = pd.read_csv('Data/InsuranceCompanyABC.csv')
cde = pd.read_csv('Data/InvestmentBankCDE.csv')
efg = pd.read_csv('Data/RetailBankEFG.csv')
answers = pd.read_csv('Data/ANSWERS.csv')

In [3]:
abc.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial
0,71504.00,42.99,4428.51,NE,1.00,T,T,T,T
1,59751.00,26.37,4489.93,CO,0.00,T,T,T,T


In [4]:
cde.head(2)

,ID,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto
0,71504.00,F,F,F,F,F,F,F
1,59751.00,F,F,F,F,F,F,F


In [5]:
efg.head(2)

,ID,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
0,71504.00,T,T,T,T,T,F,F,T,F
1,59751.00,T,T,T,T,T,F,F,F,T


In [6]:
df1 = pd.merge(cde, efg, on='ID')
data = pd.merge(abc, df1, on='ID')
data.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
0,71504.00,42.99,4428.51,NE,1.00,T,T,T,T,F,F,F,F,F,F,F,T,T,T,T,T,F,F,T,F
1,59751.00,26.37,4489.93,CO,0.00,T,T,T,T,F,F,F,F,F,F,F,T,T,T,T,T,F,F,F,T


In [7]:
answers.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.00,40.69,8474.69,CO,1.00,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
1,1028.00,56.69,7996.35,SE,0.00,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# exploration = pd.DataFrame({
#     'column': data.columns, 'type': data.dtypes, 'NA #': data.isna().sum(), 'NA %': round((data.isna().sum() / data.shape[0]) * 100, 2)
# })
# exploration.sort_values('NA %', ascending=False)

In [9]:
onehot = lambda x: 1 if x == 'T' else 0

In [10]:
for cols in data.columns[5::]:
    data[cols] = data[cols].apply(onehot)

In [11]:
# Building the model
frq_items = apriori(data.iloc[:, 5::], min_support=0.1, use_colnames=True,)

frq_items
 
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="confidence", min_threshold=0.8)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
48,"(seguro vida Emp, seguro vida PF, seguro auto)",(Seguro Residencial),0.20,0.33,0.20,1.00,3.00,0.13,361.93
97,"(Investimento LCA, Investimento Poupanca, Inve...",(Investimento Fundos_commodities),0.12,0.35,0.12,0.99,2.87,0.08,78.58
46,"(Seguro Residencial, seguro vida Emp, seguro a...",(seguro vida PF),0.20,0.25,0.20,0.99,3.96,0.15,78.79
45,"(Seguro Residencial, seguro vida Emp, seguro v...",(seguro auto),0.20,0.28,0.20,0.99,3.54,0.14,56.47
100,"(Investimento LCA, Investimento Poupanca, Inve...",(Investimento LCI),0.12,0.34,0.12,0.99,2.93,0.08,50.94
...,...,...,...,...,...,...,...,...,...
81,"(Investimento Fundos, Investimento CDB, Invest...",(Investimento LCI),0.13,0.34,0.11,0.81,2.41,0.06,3.50
4,(Emprestimo _educacao),(Investimento Fundos),0.16,0.34,0.13,0.80,2.36,0.07,3.35
33,"(Investimento Fundos, Investimento LCI)",(Investimento CDB),0.16,0.49,0.13,0.80,1.62,0.05,2.57
39,"(Investimento Fundos, Emprestimo _viagem)",(Emprestimo _educacao),0.15,0.16,0.12,0.80,5.01,0.09,4.23


In [12]:
rules['lenght'] = rules['antecedents'].apply(lambda x: len(x))

In [13]:
rules['lenght'].value_counts()

3    54
2    49
4     9
1     6
Name: lenght, dtype: int64

In [14]:
rules = rules[(rules['lenght'] > 1) & (rules['confidence'] >= 0.8)].reset_index(drop=True)

In [15]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,lenght
0,"(seguro vida Emp, seguro vida PF, seguro auto)",(Seguro Residencial),0.20,0.33,0.20,1.00,3.00,0.13,361.93,3
1,"(Investimento LCA, Investimento Poupanca, Inve...",(Investimento Fundos_commodities),0.12,0.35,0.12,0.99,2.87,0.08,78.58,4
2,"(Seguro Residencial, seguro vida Emp, seguro a...",(seguro vida PF),0.20,0.25,0.20,0.99,3.96,0.15,78.79,3
3,"(Seguro Residencial, seguro vida Emp, seguro v...",(seguro auto),0.20,0.28,0.20,0.99,3.54,0.14,56.47,3
4,"(Investimento LCA, Investimento Poupanca, Inve...",(Investimento LCI),0.12,0.34,0.12,0.99,2.93,0.08,50.94,4
...,...,...,...,...,...,...,...,...,...,...
107,"(Investimento LCA, Investimento Fundos_commodi...","(Investimento Fundos_cambiais, Investimento LCI)",0.16,0.20,0.13,0.81,4.01,0.10,4.25,2
108,"(Investimento CDB, seguro vida PF)",(seguro auto),0.12,0.28,0.10,0.81,2.91,0.07,3.83,2
109,"(Investimento Fundos, Investimento CDB, Invest...",(Investimento LCI),0.13,0.34,0.11,0.81,2.41,0.06,3.50,3
110,"(Investimento Fundos, Investimento LCI)",(Investimento CDB),0.16,0.49,0.13,0.80,1.62,0.05,2.57,2


In [16]:
rules.groupby(['antecedents', 'consequents']).agg({'confidence': 'max'}).sort_values('confidence', ascending=False)

,,confidence
antecedents,consequents,
"(seguro vida Emp, seguro vida PF, seguro auto)",(Seguro Residencial),1.00
"(Investimento LCA, Investimento Poupanca, Investimento Fundos_cambiais, Investimento LCI)",(Investimento Fundos_commodities),0.99
"(Seguro Residencial, seguro vida Emp, seguro auto)",(seguro vida PF),0.99
"(Seguro Residencial, seguro vida Emp, seguro vida PF)",(seguro auto),0.99
"(Investimento LCA, Investimento Poupanca, Investimento Fundos_cambiais, Investimento Fundos_commodities)",(Investimento LCI),0.99
...,...,...
"(Investimento LCA, Investimento Fundos_commodities)","(Investimento Fundos_cambiais, Investimento LCI)",0.81
"(Investimento CDB, seguro vida PF)",(seguro auto),0.81
"(Investimento Fundos, Investimento CDB, Investimento Fundos_cambiais)",(Investimento LCI),0.81


In [29]:
for row1, row2 in zip(rules.itertuples(index=False), answers.itertuples(index=False)):
    print(row, row2)
    break

Pandas(antecedents=frozenset({'seguro vida Emp', 'seguro vida PF', 'seguro auto'}), consequents=frozenset({'Seguro Residencial'}), _2=0.19745083758193735, _3=0.33248361252731246, support=0.19708667152221412, confidence=0.9981556621172999, lift=3.002119877518188, leverage=0.13143750374642796, conviction=361.9273852876915, lenght=3) Pandas(ID=1004.0, Idade=40.685, Renda=8474.687, Regiao='CO', Genero=1.0, _5='F', _6='F', _7='F', _8='F', _9='F', _10='T', _11='F', _12='F', _13='F', _14='T', _15='F', _16='F', _17='F', _18='F', _19='F', _20='F', _21='T', _22='F', _23='T', _24='F', RECOMMENDATION_1=nan, CONFIDENCE_1=nan, RECOMMENDATION_2=nan, CONFIDENCE_2=nan, RECOMMENDATION_3=nan, CONFIDENCE_3=nan)


In [27]:
row.antecedents & row.consequents

frozenset()

In [17]:
set(services[0]) & set(rules['antecedents'][0])

NameError: name 'services' is not defined

In [ ]:
answers.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.00,40.69,8474.69,CO,1.00,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
1,1028.00,56.69,7996.35,SE,0.00,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# index = []
# aux = []
# services = []
# for val in range(data.shape[0]):
#     index.append(answers.index[val])
#     for cols in data.columns[5::]:
#         if answers[cols][val] == 'T':
#             aux.append(cols)
#         else: continue
#     services.append(aux)
#     aux = []

In [36]:
for row1, row2 in zip(data.itertuples(index=False), services.itertuples(index=False)):
    print(row1)
    break

Pandas(ID=71504.0, Idade=42.993, Renda=4428.512, Regiao='NE', Genero=1.0, _5=1, _6=1, _7=1, _8=1, _9=0, _10=0, _11=0, _12=0, _13=0, _14=0, _15=0, _16=1, _17=1, _18=1, _19=1, _20=1, _21=0, _22=0, _23=1, _24=0)


In [ ]:
# set(services[0]) & set(rules['antecedents'][0])

set()

In [ ]:
# # recomendations = pd.DataFrame({'index': index, 'services': services})
# confidence = []
# conf = []
# index = []
# for serv in range(len(services)):
#     index.append(answers.index[serv])
#     for ant in range(rules.shape[0]):
#         if set(services[serv]) & set(rules['antecedents'][ant]):
#             conf.append(rules['confidence'][ant])
#         else: continue
#     confidence.append(conf)
#     conf = []

In [ ]:
len(confidence)

13730

In [ ]:
index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
